# Structured Data Assignment

## Problem Statement

The dataset in question contains a comprehensive collection of electronic health records belonging to patients who have been diagnosed with a specific disease. These health records comprise a detailed log of every aspect of the patients' medical history, including all diagnoses, symptoms, prescribed drug treatments, and medical tests that they have undergone. Each row represents a healthcare record/medical event for a patient and it includes a timestamp for each entry/event, thereby allowing for a chronological view of the patient's medical history.

The Data has mainly three columns

   1. Patient-Uid - Unique Alphanumeric Identifier for a patient
   2. Date - Date when patient encountered the event.
   3. Incident - This columns describes which event occurred on the day.

## Problem

The development of drugs is critical in providing therapeutic options for patients suffering from chronic and terminal illnesses. “Target Drug”, in particular, is designed to enhance the patient's health and well-being without causing dependence on other medications that could potentially lead to severe and life-threatening side effects. These drugs are specifically tailored to treat a particular disease or condition, offering a more focused and effective approach to treatment, while minimising the risk of harmful reactions.

## Objective

To develop a predictive model which will predict whether a patient will be eligible*** for “Target Drug” or not in next 30 days. Knowing if the patient is eligible or not will help physician treating the patient make informed decision on the which treatments to give.

A patient is considered eligible for a particular drug when they have taken their first prescription for that drug.


<----------------------------------------------------------------------------------------------------------------------------->


In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.exceptions import ConvergenceWarning
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

In [2]:

df_train = pd.read_parquet(r"E:\downloads\assignment\Structured_Data_Assignment\train.parquet")

In [3]:

df_train.head()

,Patient-Uid,Date,Incident
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2019-03-09,PRIMARY_DIAGNOSIS
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,2015-05-16,PRIMARY_DIAGNOSIS
3,a0dc94c6-1c7c-11ec-a3a0-16262ee38c7f,2018-01-30,SYMPTOM_TYPE_0
4,a0dc950b-1c7c-11ec-b6ec-16262ee38c7f,2015-04-22,DRUG_TYPE_0
8,a0dc9543-1c7c-11ec-bb63-16262ee38c7f,2016-06-18,DRUG_TYPE_1


In [4]:

df_train.shape

(3220868, 3)

## Data Cleaning

## Missing Values

In [5]:

# let's check whether there is any missing values present in the dataset
# here itself let's check the dtypes of the columns

warnings.simplefilter(action='ignore',category=FutureWarning)

col_info = pd.DataFrame(df_train.dtypes).T.rename(index = {0:'Column Types'})

col_info = col_info.append(pd.DataFrame(df_train.isnull().sum()).T.rename(index = {0:'Null Values'}))

col_info = col_info.append(pd.DataFrame((df_train.isnull().sum())/len(df_train)).T.rename(index = {0:'Null Values (%)'}))
col_info

,Patient-Uid,Date,Incident
Column Types,object,datetime64[ns],object
Null Values,0,0,0
Null Values (%),0.0,0.0,0.0


## Duplicates

In [6]:

df_train.duplicated().sum()

35571

In [7]:

# let's drop the dublicates from the datasets

df_train = df_train.drop_duplicates()
df_train.duplicated().sum()

0

## Misspelled word

In [8]:
print("Unique values of Incident \n")
print(df_train['Incident'].unique())

Unique values of Incident 

['PRIMARY_DIAGNOSIS' 'SYMPTOM_TYPE_0' 'DRUG_TYPE_0' 'DRUG_TYPE_1'
 'DRUG_TYPE_2' 'TEST_TYPE_0' 'DRUG_TYPE_3' 'DRUG_TYPE_4' 'DRUG_TYPE_5'
 'DRUG_TYPE_6' 'DRUG_TYPE_8' 'DRUG_TYPE_7' 'SYMPTOM_TYPE_1' 'DRUG_TYPE_10'
 'SYMPTOM_TYPE_29' 'SYMPTOM_TYPE_2' 'DRUG_TYPE_11' 'DRUG_TYPE_9'
 'DRUG_TYPE_13' 'SYMPTOM_TYPE_5' 'TEST_TYPE_1' 'SYMPTOM_TYPE_6'
 'TEST_TYPE_2' 'SYMPTOM_TYPE_3' 'SYMPTOM_TYPE_8' 'DRUG_TYPE_14'
 'DRUG_TYPE_12' 'SYMPTOM_TYPE_9' 'SYMPTOM_TYPE_10' 'SYMPTOM_TYPE_7'
 'SYMPTOM_TYPE_11' 'TEST_TYPE_3' 'DRUG_TYPE_15' 'SYMPTOM_TYPE_4'
 'SYMPTOM_TYPE_14' 'SYMPTOM_TYPE_13' 'SYMPTOM_TYPE_16' 'SYMPTOM_TYPE_17'
 'SYMPTOM_TYPE_15' 'SYMPTOM_TYPE_18' 'SYMPTOM_TYPE_12' 'SYMPTOM_TYPE_20'
 'SYMPTOM_TYPE_21' 'DRUG_TYPE_17' 'SYMPTOM_TYPE_22' 'TEST_TYPE_4'
 'SYMPTOM_TYPE_23' 'DRUG_TYPE_16' 'TEST_TYPE_5' 'SYMPTOM_TYPE_19'
 'SYMPTOM_TYPE_24' 'SYMPTOM_TYPE_25' 'SYMPTOM_TYPE_26' 'SYMPTOM_TYPE_27'
 'DRUG_TYPE_18' 'SYMPTOM_TYPE_28' 'TARGET DRUG']


## Exploring The Dataset

In [9]:

#no of counts for particular Incident

df_train.Incident.value_counts()

DRUG_TYPE_6          549616
DRUG_TYPE_1          484565
PRIMARY_DIAGNOSIS    424879
DRUG_TYPE_0          298881
DRUG_TYPE_2          256203
DRUG_TYPE_7          251239
DRUG_TYPE_8          158706
DRUG_TYPE_3          126615
TEST_TYPE_1           96810
TARGET DRUG           67218
DRUG_TYPE_9           66894
DRUG_TYPE_5           55940
SYMPTOM_TYPE_0        46078
DRUG_TYPE_11          45419
SYMPTOM_TYPE_6        32066
TEST_TYPE_0           27570
SYMPTOM_TYPE_7        22019
DRUG_TYPE_10          20911
DRUG_TYPE_14          17306
DRUG_TYPE_13          12321
DRUG_TYPE_12           9540
SYMPTOM_TYPE_14        8927
SYMPTOM_TYPE_1         8608
SYMPTOM_TYPE_2         8168
TEST_TYPE_3            8115
SYMPTOM_TYPE_5         7583
SYMPTOM_TYPE_8         7430
TEST_TYPE_2            7021
SYMPTOM_TYPE_15        6295
SYMPTOM_TYPE_10        6005
SYMPTOM_TYPE_29        5910
SYMPTOM_TYPE_16        4940
DRUG_TYPE_15           4906
SYMPTOM_TYPE_9         4885
DRUG_TYPE_4            4566
SYMPTOM_TYPE_4      

In [10]:

#no of counts for particular Date

df_train.Date.value_counts()

2019-05-21    3645
2019-05-22    3330
2019-03-05    3156
2019-05-14    3134
2019-03-12    2986
              ... 
2017-01-27     308
2017-02-28     285
2020-02-29     282
2016-01-29     271
2016-02-29     255
Name: Date, Length: 1977, dtype: int64

In [11]:

# let's assume TARGET DRUG as df_positive

df_positive = df_train[df_train['Incident']=='TARGET DRUG']
df_positive.head()

,Patient-Uid,Date,Incident
3294791,a0eb742b-1c7c-11ec-8f61-16262ee38c7f,2020-04-09,TARGET DRUG
3296990,a0edaf09-1c7c-11ec-a360-16262ee38c7f,2018-06-12,TARGET DRUG
3305387,a0e9fa0e-1c7c-11ec-8dc7-16262ee38c7f,2019-06-11,TARGET DRUG
3309423,a0ecc615-1c7c-11ec-aa31-16262ee38c7f,2019-11-15,TARGET DRUG
3309494,a0ea612f-1c7c-11ec-8cf0-16262ee38c7f,2020-03-18,TARGET DRUG


In [12]:

# shape of positive set

df_positive.shape

(67218, 3)

In [13]:

# let's assume other than TARGET DRUG as df_negative

negative =  df_train[~df_train['Patient-Uid'].isin(df_positive['Patient-Uid'])]
df_negative = negative.groupby('Patient-Uid').tail(1)
df_negative

,Patient-Uid,Date,Incident
1560892,a0e3a8c0-1c7c-11ec-98c2-16262ee38c7f,2018-05-06,PRIMARY_DIAGNOSIS
1620903,a0dd6a3f-1c7c-11ec-9b86-16262ee38c7f,2015-04-07,SYMPTOM_TYPE_0
1629044,a0e48a75-1c7c-11ec-8c5f-16262ee38c7f,2018-08-22,DRUG_TYPE_6
1942882,a0e3cf61-1c7c-11ec-8098-16262ee38c7f,2018-08-21,DRUG_TYPE_2
1975541,a0e91a8c-1c7c-11ec-acc2-16262ee38c7f,2020-04-15,PRIMARY_DIAGNOSIS
...,...,...,...
3256795,a0e045a1-1c7c-11ec-8014-16262ee38c7f,2020-07-10,PRIMARY_DIAGNOSIS
3256799,a0e67e2a-1c7c-11ec-b805-16262ee38c7f,2015-12-16,PRIMARY_DIAGNOSIS
3256800,a0dec400-1c7c-11ec-80df-16262ee38c7f,2019-08-06,PRIMARY_DIAGNOSIS
3256804,a0e09919-1c7c-11ec-9e7d-16262ee38c7f,2017-02-19,DRUG_TYPE_6


In [14]:

# shape of negative set

df_negative.shape

(17659, 3)

In [15]:

# to get the count of previous prescriptions within specific time intervals

df_positive = df_positive.copy()
df_negative = df_negative.copy()
df_positive['Prescription_Count'] = df_positive.groupby('Patient-Uid')['Date'].cumcount()
df_negative['Prescription_Count'] = df_negative.groupby('Patient-Uid')['Date'].cumcount()
df_positive.tail(5)

,Patient-Uid,Date,Incident,Prescription_Count
29074998,a0ef2b6d-1c7c-11ec-9172-16262ee38c7f,2018-10-12,TARGET DRUG,4
29075105,a0ebe423-1c7c-11ec-a5e0-16262ee38c7f,2019-07-02,TARGET DRUG,9
29075494,a0ebc713-1c7c-11ec-bd53-16262ee38c7f,2019-05-21,TARGET DRUG,10
29080031,a0ee1bdb-1c7c-11ec-90ba-16262ee38c7f,2018-06-07,TARGET DRUG,14
29080178,a0eef180-1c7c-11ec-8de8-16262ee38c7f,2018-07-17,TARGET DRUG,13


In [16]:
df_negative.tail()

,Patient-Uid,Date,Incident,Prescription_Count
3256795,a0e045a1-1c7c-11ec-8014-16262ee38c7f,2020-07-10,PRIMARY_DIAGNOSIS,0
3256799,a0e67e2a-1c7c-11ec-b805-16262ee38c7f,2015-12-16,PRIMARY_DIAGNOSIS,0
3256800,a0dec400-1c7c-11ec-80df-16262ee38c7f,2019-08-06,PRIMARY_DIAGNOSIS,0
3256804,a0e09919-1c7c-11ec-9e7d-16262ee38c7f,2017-02-19,DRUG_TYPE_6,0
3256805,a0e69331-1c7c-11ec-a98d-16262ee38c7f,2015-10-03,DRUG_TYPE_6,0


In [17]:

# to get the difference between the most recent prescription and the prediction date.

prediction_date = pd.to_datetime('today') + pd.DateOffset(days=30)
df_positive['Time_diff'] = (prediction_date - df_positive.groupby('Patient-Uid')['Date'].transform('max')).dt.days
df_negative['Time_diff'] = (prediction_date - df_negative.groupby('Patient-Uid')['Date'].transform('max')).dt.days
df_positive.head()

,Patient-Uid,Date,Incident,Prescription_Count,Time_diff
3294791,a0eb742b-1c7c-11ec-8f61-16262ee38c7f,2020-04-09,TARGET DRUG,0,1195
3296990,a0edaf09-1c7c-11ec-a360-16262ee38c7f,2018-06-12,TARGET DRUG,0,1449
3305387,a0e9fa0e-1c7c-11ec-8dc7-16262ee38c7f,2019-06-11,TARGET DRUG,0,1474
3309423,a0ecc615-1c7c-11ec-aa31-16262ee38c7f,2019-11-15,TARGET DRUG,0,1179
3309494,a0ea612f-1c7c-11ec-8cf0-16262ee38c7f,2020-03-18,TARGET DRUG,0,1195


In [18]:
df_negative.head()

,Patient-Uid,Date,Incident,Prescription_Count,Time_diff
1560892,a0e3a8c0-1c7c-11ec-98c2-16262ee38c7f,2018-05-06,PRIMARY_DIAGNOSIS,0,2025
1620903,a0dd6a3f-1c7c-11ec-9b86-16262ee38c7f,2015-04-07,SYMPTOM_TYPE_0,0,3150
1629044,a0e48a75-1c7c-11ec-8c5f-16262ee38c7f,2018-08-22,DRUG_TYPE_6,0,1917
1942882,a0e3cf61-1c7c-11ec-8098-16262ee38c7f,2018-08-21,DRUG_TYPE_2,0,1918
1975541,a0e91a8c-1c7c-11ec-acc2-16262ee38c7f,2020-04-15,PRIMARY_DIAGNOSIS,0,1315


In [23]:
df = pd.concat([df_positive, df_negative])
df.head()

,Patient-Uid,Date,Incident,Prescription_Count,Time_diff
3294791,a0eb742b-1c7c-11ec-8f61-16262ee38c7f,2020-04-09,TARGET DRUG,0,1195
3296990,a0edaf09-1c7c-11ec-a360-16262ee38c7f,2018-06-12,TARGET DRUG,0,1449
3305387,a0e9fa0e-1c7c-11ec-8dc7-16262ee38c7f,2019-06-11,TARGET DRUG,0,1474
3309423,a0ecc615-1c7c-11ec-aa31-16262ee38c7f,2019-11-15,TARGET DRUG,0,1179
3309494,a0ea612f-1c7c-11ec-8cf0-16262ee38c7f,2020-03-18,TARGET DRUG,0,1195


In [24]:

df.shape

(84877, 5)

In [25]:

# let's set 1 for TARGET DRUG and 0 for other than TARGET DRUG

df['Target'] = np.where(df['Incident'] == 'TARGET DRUG', 1, 0)

In [26]:

df.head()

,Patient-Uid,Date,Incident,Prescription_Count,Time_diff,Target
3294791,a0eb742b-1c7c-11ec-8f61-16262ee38c7f,2020-04-09,TARGET DRUG,0,1195,1
3296990,a0edaf09-1c7c-11ec-a360-16262ee38c7f,2018-06-12,TARGET DRUG,0,1449,1
3305387,a0e9fa0e-1c7c-11ec-8dc7-16262ee38c7f,2019-06-11,TARGET DRUG,0,1474,1
3309423,a0ecc615-1c7c-11ec-aa31-16262ee38c7f,2019-11-15,TARGET DRUG,0,1179,1
3309494,a0ea612f-1c7c-11ec-8cf0-16262ee38c7f,2020-03-18,TARGET DRUG,0,1195,1


# Splitting For Training And Testing

In [27]:

from sklearn.model_selection import train_test_split

X = df[['Prescription_Count','Time_diff']].values
y = df['Target'].values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [28]:

X_train.shape,X_test.shape,y_train.shape,y_test.shape

((67901, 2), (16976, 2), (67901,), (16976,))

# MODEL SELECTION

## Random Forest Classifier

In [ ]:

from sklearn.ensemble import RandomForestClassifier

rf_class = RandomForestClassifier(random_state=42)
rf_class.fit(X_train, y_train)

In [ ]:

#predicting test data

y_pred = rf_class.predict(X_test)

In [ ]:

# to evaluate the model - confusion_matrix

from sklearn.metrics import confusion_matrix
conf_matrix_report = confusion_matrix(y_test, y_pred)
conf_matrix_report

In [ ]:

# evaluating the model - classification report

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:

# Importing the evaluation metrics

from sklearn.metrics import f1_score, roc_auc_score

In [ ]:

# f1 score for RF

rf_f1_score = f1_score(y_test, y_pred)
print("F1 score for RF classification model :",rf_f1_score)

In [ ]:

# auroc score for RF model

rf_auroc_score = roc_auc_score(y_test, y_pred)
print("AUROC score for RF classification model :",rf_auroc_score)

## XGBoost Classifier

In [ ]:

import xgboost as xgb

xgb_class = xgb.XGBClassifier(random_state=42)
xgb_class.fit(X_train, y_train)

In [ ]:

#predicting test data

y_pred = xgb_class.predict(X_test)

In [ ]:

# to evaluate the model - confusion_matrix

conf_matrix_report = confusion_matrix(y_test, y_pred)
conf_matrix_report

In [ ]:

# evaluating the model - classification report

print(classification_report(y_test, y_pred))

In [ ]:

# f1 score for RF

xgb_f1_score = f1_score(y_test, y_pred)
print("F1 score for XGB classification model :",xgb_f1_score)

In [ ]:

# auroc score for RF model

xgb_auroc_score = roc_auc_score(y_test, y_pred)
print("AUROC score for XGB classification model :",xgb_auroc_score)

## EVALUATION METRICS

### F1 SCORE

In [ ]:
print('F1 Score For RF Classification model        :',rf_f1_score)
print('F1 Score For XGBOOST Classification model   :',xgb_f1_score)

### AUROC SCORE

In [ ]:
print('AUROC Score For RF Classification model        :',rf_auroc_score)
print('AUROC Score For XGBOOST Classification model   :',xgb_auroc_score)

# Loadind The Test Dataset

In [ ]:

df_test = pd.read_parquet(r"E:\downloads\assignment\Structured_Data_Assignment\test.parquet")
df_test.head()

In [ ]:
df_test.shape

## Data Cleaning

In [ ]:

warnings.simplefilter(action='ignore',category=FutureWarning)

col_info = pd.DataFrame(df_test.dtypes).T.rename(index = {0:'Column Types'})

col_info = col_info.append(pd.DataFrame(df_test.isnull().sum()).T.rename(index = {0:'Null Values'}))

col_info = col_info.append(pd.DataFrame((df_test.isnull().sum())/len(df_test)).T.rename(index = {0:'Null Values (%)'}))
col_info

In [ ]:

# let's check is there is any duplicates 

df_test.duplicated().sum()

In [ ]:

# let's drop the dublicates from the datasets

df_test = df_test.drop_duplicates()
df_test.duplicated().sum()

In [ ]:

# let's assume TARGET DRUG as df_positive

df_positive = df_test[df_test['Incident']=='TARGET DRUG']
df_positive.head()

In [ ]:

# where there is no TARGET DRUG present in the Dataset
# shape of positive set

df_positive.shape

In [ ]:

# let's assume other than TARGET DRUG as df_negative

negative =  df_test[~df_test['Patient-Uid'].isin(df_positive['Patient-Uid'])]
df_negative = negative.groupby('Patient-Uid').tail(1)
df_negative

In [ ]:

# shape of negative set

df_negative.shape

In [ ]:

# to get the count of previous prescriptions within specific time intervals

df_positive = df_positive.copy()
df_negative = df_negative.copy()
df_positive['Prescription_Count'] = df_positive.groupby('Patient-Uid')['Date'].cumcount()
df_negative['Prescription_Count'] = df_negative.groupby('Patient-Uid')['Date'].cumcount()

In [ ]:

# to get the difference between the most recent prescription and the prediction date.

prediction_date = pd.to_datetime('today') + pd.DateOffset(days=30)
df_positive['Time_diff'] = (prediction_date - df_positive.groupby('Patient-Uid')['Date'].transform('max')).dt.days
df_negative['Time_diff'] = (prediction_date - df_negative.groupby('Patient-Uid')['Date'].transform('max')).dt.days
df_positive.head()

In [ ]:
df_negative.head()

In [ ]:

# creating new dataset by concating positive and negative sets

df = pd.concat([df_positive, df_negative])
df.head()

In [ ]:
df.shape

In [ ]:

test_data_pred = xgb_class.predict(df[['Prescription_Count', 'Time_diff']])

In [ ]:
test_data_pred

In [ ]:

# to create final submission file

Final_submission = pd.DataFrame({'Patient-Uid': df['Patient-Uid'], 'Prediction': test_data_pred})
Final_submission.head()

In [ ]:

Final_submission.to_csv('Final_submission.csv', index = False)